In [7]:
from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)

# Load the dataset once at the start
df = pd.read_csv('Documents/hotel_bookings_1000.csv')

# Clean and preprocess the data
df['arrival_date'] = pd.to_datetime(df['arrival_date_year'].astype(str) + '-' +
                                    df['arrival_date_month'] + '-' +
                                    df['arrival_date_day_of_month'].astype(str))

@app.route('/get-data', methods=['GET'])
def get_data():
    # Get date range from query parameters
    start_date = request.args.get('start_date')
    end_date = request.args.get('end_date')

    # Filter the dataframe by the date range
    if start_date and end_date:
        filtered_data = df[(df['arrival_date'] >= start_date) & (df['arrival_date'] <= end_date)]
    else:
        filtered_data = df

    # Prepare the data for the charts
    # 1. Time Series: Sum of visitors (adults + children + babies) per day
    time_series = filtered_data.groupby('arrival_date').agg(
        visitors=pd.NamedAgg(column='adults', aggfunc='sum') +
                 pd.NamedAgg(column='children', aggfunc='sum') +
                 pd.NamedAgg(column='babies', aggfunc='sum')
    ).reset_index().to_dict(orient='records')

    # 2. Country-wise Visitors
    country_data = filtered_data.groupby('country').size().reset_index(name='visitors').to_dict(orient='records')

    # 3. Sparklines Data
    adults_data = filtered_data.groupby('arrival_date')['adults'].sum().tolist()
    children_data = filtered_data.groupby('arrival_date')['children'].sum().tolist()

    # Return the results as JSON
    return jsonify({
        'time_series': time_series,
        'countries': country_data,
        'adults': adults_data,
        'children': children_data
    })

if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\supri\miniforge3\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
